In [33]:
import os, h5py, shutil
import zipfile
import numpy as np
import subprocess as sp
import pandas as pd
import matplotlib.pyplot as plt
from skimage.util.shape import view_as_blocks

In [12]:
###----------------------user setting here

filenum=0 #there are 6 total, use numbers 0 through 5
CAMPAIGN='Grnlnd' #or 'NISARA' for AM and 'NISARP' for PM
SITE='00005'
PROD='129' #129 = global 20 & 5 MHz, 138 = US Ag 40 & 5 MHz
PROD2='CG' #CX = normal, CG = dithered-gaps, CD = dithered-no-gaps
channels='A' #A is (HH,HV) and has higher frequency compared to B (20 or 40 MHz vs 5 or 20 MHz); whereas B is (VH,VV). PROD='143' has both channels at 20 MHz
pol = 'HH'
# symmetrize = 1 #use UAVSAR's approach for symmetrizing HV and VH
#symmetrization: based on the principle of reciprocity HV should give the same result of VH. But due to instrument limitations (e.g. thermal noise/channel imbalance/cable lengths)
#vh and hv will not match in practice. The symmetrization is a statistical approach for reducing the imbalance, where the entire image statistics are used as inputs
#to determine correction factors for magnitude and phase for this imbalance. As result, the HV and VH channels will become much more similar on the whole; but there may still be some clear
#differences at the level of individual pixels.

In [13]:
fnr = [f for f in os.listdir('.') if f.endswith('.h5')][filenum]#here: 'Grnlnd_00005_09030_007_090527_L090_CG_129_03.h5'
fnr

'Grnlnd_00005_09030_007_090527_L090_CG_129_03.h5'

In [14]:
n_HH = 'science/LSAR/SLC/swaths/frequency'+channels+'/HH'
    #only testing HH, no symmetrisation needed
# n_HV = 'science/LSAR/SLC/swaths/frequency'+channels+'/HV'
# n_VH = 'science/LSAR/SLC/swaths/frequency'+channels+'/VH'
# n_VV = 'science/LSAR/SLC/swaths/frequency'+channels+'/VV'

with h5py.File(fnr, 'r') as f:  
    dat_HH = f[n_HH][()]
    print(dat_HH.shape)

(28942, 2640)


In [16]:
#write the slc array data
aa = os.path.split(fnr)[1].split('_'+PROD2)
basen_s = aa[0]
basen_e = aa[1]
fno = basen_s + pol +'_CX'+basen_e[:-2]+'slc'
fno = fno.replace('_'+PROD, '_'+PROD+channels)

dat_HH.tofile(fno)

In [29]:
#take 'cross product'
dat_HHHH = (dat_HH*dat_HH.conjugate()).astype(np.float32)

D:\Anaconda3\envs\cf\lib\site-packages\ipykernel_launcher.py:2: ComplexWarning: Casting complex values to real discards the imaginary part
  


In [42]:
dat_HHHH

array([[2.2712359e-04, 2.2217163e-04, 1.3303758e-04, ..., 2.1639757e-04,
        2.0540999e-04, 4.5840152e-05],
       [4.2941788e-04, 1.2820647e-03, 1.1318357e-03, ..., 1.2123203e-03,
        3.4086962e-04, 5.6789716e-04],
       [2.0697970e-04, 2.3592707e-04, 9.6793758e-04, ..., 9.4139083e-05,
        6.2010229e-05, 2.6249481e-04],
       ...,
       [1.1744055e-04, 1.3135451e-04, 1.0355081e-04, ..., 1.9424166e-05,
        1.7257403e-04, 1.3536782e-04],
       [1.6714798e-03, 8.7150279e-04, 9.1353548e-05, ..., 1.7130644e-04,
        8.6226471e-04, 3.5169264e-04],
       [5.0896063e-04, 4.2180961e-04, 8.8740664e-04, ..., 7.7453878e-04,
        1.6740111e-04, 6.5280049e-04]], dtype=float32)

In [43]:
#multi looking. 
#If want to feed these data to the ISCE RTC procesing:
#Check the .ann file to see the number of row and colums the MLC is supposed to have. Then multi look accordingly. If needed skip a row or column.
#note: alooks = 2, and if prod is 138 rlooks = 2. Else rlooks = 1. 
#the ann shows for mlc_pwr.set_rows = 14471 and  mlc_pwr.set_cols =2640 cols
#dat_HH.shape shows (28942, 2640), so this works out without needing to skip any rows or columns

In [49]:
view = view_as_blocks(dat_HHHH, (2, 1)) 
flatten_view = view.reshape(view.shape[0], view.shape[1], -1)
mlook_HHHH = np.mean(flatten_view, axis=2)

In [51]:
mlook_HHHH

array([[3.2827072e-04, 7.5211818e-04, 6.3243665e-04, ..., 7.1435893e-04,
        2.7313980e-04, 3.0686866e-04],
       [3.2200609e-04, 7.2697707e-04, 5.4426148e-04, ..., 7.5848959e-04,
        2.3122547e-04, 6.7627890e-04],
       [3.3598513e-04, 9.8956494e-05, 5.6545518e-04, ..., 7.9105352e-04,
        3.1839398e-04, 6.2175663e-05],
       ...,
       [9.8057045e-04, 8.6992770e-04, 3.8915282e-04, ..., 3.9476281e-04,
        1.2730164e-03, 5.4349360e-04],
       [5.0977350e-04, 3.2812802e-04, 1.2775415e-04, ..., 3.1113723e-05,
        1.3174534e-04, 1.5907839e-04],
       [1.0902202e-03, 6.4665620e-04, 4.8938009e-04, ..., 4.7292261e-04,
        5.1483291e-04, 5.0224655e-04]], dtype=float32)

In [53]:
#check math
chk = (dat_HHHH[0,0]+dat_HHHH[1,0])/2  #averaging in azimuth: averaging pixels along rows. Here the first pixel in first row, with first pixel in second row 
print(chk)
print(mlook_HHHH[0,0]) #truncated because I use float32 instead of float64

0.00032827071845531464
0.00032827072


In [54]:
mlook_HHHH.shape

(14471, 2640)

In [58]:
#write the mlc array data
aa = os.path.split(fnr)[1].split('_'+PROD2)
basen_s = aa[0]
basen_e = aa[1]
fno = basen_s + pol +'_CX'+basen_e[:-2]+'mlc'
fno = fno.replace('_'+PROD, '_'+PROD+channels)
fno = fno.replace('HH', 'HHHH')

mlook_HHHH.tofile(fno)

In [60]:
#function for writing the vrt
def genvrt(fni, cl, rw, lo):
    tmp = """<VRTDataset rasterXSize='{cl}' rasterYSize='{rw}'>
    <VRTRasterBand band="1" dataType="Float32" subClass="VRTRawRasterBand">
        <SourceFilename relativeToVRT="1">{fni}</SourceFilename>
        <ByteOrder>LSB</ByteOrder>
        <ImageOffset>0</ImageOffset>
        <PixelOffset>4</PixelOffset>
        <LineOffset>{lo}</LineOffset>
    </VRTRasterBand>
</VRTDataset>"""

    context = {"cl":cl, "rw":rw, "fni":fni, "lo":lo}
    with open(fni+'.vrt', 'w') as f:
        f.write(tmp.format(**context))

In [61]:
genvrt(fno, 2640, 14471, mlook_HHHH.shape[1]*4)

In [ ]:
#for comparison check the qgis file compare_noISCE_toISCE. It is important to note that ISCE attaches some projection information to the vrt, you should delete those lines before comparing:
#<SRS>EPSG:4326</SRS>
#<GeoTransform>0.0, 1.0, 0.0, 0.5, 0.0, 2.0</GeoTransform>
#I see no difference. All array entries are 0.